In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from math import sqrt
import math
import os
import json
import time
import seaborn as sns
import datetime
from core.data_processor import DataLoader
from core.model import Model
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils


Using TensorFlow backend.


In [2]:
'''
till now judgement
date_time: convert to suitable format, doesnt go for prediction 
dayofweek: add new column signifying day(Monday=0, Sunday=6)
peakhours: 
is_holiday : change all non null to 1
wind_speed : to be standardized variable
wind_direction: categorical variable, better to put in 60 deg buckets 
visibility_in_miles: categorical
dew_point: categorical
temperature: replace 0 with mean value and standardize it
air_pollution_index : No change, just standardize it
humidity: standardize it
rain_p_h: have to think , for now just use as it is
snow_ph: 0 and 1
weather_type: label encode
weather_description: for now label encode
clouds_all : 
traffic_volume: to predict
'''


'\ntill now judgement\ndate_time: convert to suitable format, doesnt go for prediction \ndayofweek: add new column signifying day(Monday=0, Sunday=6)\npeakhours: \nis_holiday : change all non null to 1\nwind_speed : to be standardized variable\nwind_direction: categorical variable, better to put in 60 deg buckets \nvisibility_in_miles: categorical\ndew_point: categorical\ntemperature: replace 0 with mean value and standardize it\nair_pollution_index : No change, just standardize it\nhumidity: standardize it\nrain_p_h: have to think , for now just use as it is\nsnow_ph: 0 and 1\nweather_type: label encode\nweather_description: for now label encode\nclouds_all : \ntraffic_volume: to predict\n'

In [3]:
rawdata = pd.read_csv("./DataSets/Train.csv")
df_test = pd.read_csv("./DataSets/Test.csv")

In [4]:
rawdata.head()

,date_time,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,weather_description,traffic_volume
0,2012-10-02 09:00:00,None,121,89,2,329,1,1,288.28,0.0,0.0,40,Clouds,scattered clouds,5545
1,2012-10-02 10:00:00,None,178,67,3,330,1,1,289.36,0.0,0.0,75,Clouds,broken clouds,4516
2,2012-10-02 11:00:00,None,113,66,3,329,2,2,289.58,0.0,0.0,90,Clouds,overcast clouds,4767
3,2012-10-02 12:00:00,None,20,66,3,329,5,5,290.13,0.0,0.0,90,Clouds,overcast clouds,5026
4,2012-10-02 13:00:00,None,281,65,3,329,7,7,291.14,0.0,0.0,75,Clouds,broken clouds,4918


In [5]:
rawdata.date_time = pd.to_datetime(rawdata['date_time'])

In [6]:
def preprocessing(rawdata):
    df = rawdata.copy()
    #day of week, we have less traffic during saturdays or sundays
    df['dayofweek'] = rawdata.date_time.dt.dayofweek
    df['month'] = rawdata.date_time.dt.month
    #hours
    bins = [-0.1,6,12,18,24]
    labels = [0,1,2,3]
    df['hour']= pd.cut(rawdata['date_time'].dt.hour, bins=bins, labels=labels)
    #is_holiday
    df['is_holiday'] = (rawdata.is_holiday!='None')*1
    A = df.loc[df.is_holiday==1,'date_time']
    for items in A:
        df.loc[(df.date_time.dt.year==items.year) & (df.date_time.dt.month==items.month) & (df.date_time.dt.day==items.day),'is_holiday'] = 1
    #wind_direction
    bins = [-0.1, 60, 120, 180, 240, 300, 360]
    labels = [0,1,2,3,4,5]
    df['wind_direction']= pd.cut(rawdata['wind_direction'], bins=bins, labels=labels)
    #temperature
    df.loc[rawdata.temperature==0,'temperature'] = np.mean(rawdata.loc[rawdata.temperature!=0,'temperature'])
    #snow_p_h
    df['snow_p_h'] = (rawdata.snow_p_h!=0)*1
    #weather label encode
    #Encoding features
    lb_weather = LabelEncoder()
    df['weather_type'] = lb_weather.fit_transform(rawdata['weather_type'])
    lb_weather_des = LabelEncoder()
    df['weather_description'] = lb_weather_des.fit_transform(rawdata['weather_description'])
    #standardizing
    std_cols = ['temperature','air_pollution_index','humidity','wind_speed','clouds_all','traffic_volume']
    mean = rawdata[std_cols].mean()
    std = rawdata[std_cols].std()
    df[std_cols] = (rawdata[std_cols]-mean)/std
    return df,(lb_weather,lb_weather_des),(std_cols,mean,std)

In [7]:
df,encoders,stats = preprocessing(rawdata)

In [8]:
df.describe()

,is_holiday,air_pollution_index,humidity,wind_speed,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,weather_description,traffic_volume,dayofweek,month
count,33750.000000,3.375000e+04,3.375000e+04,3.375000e+04,33750.000000,33750.000000,3.375000e+04,33750.000000,33750.000000,3.375000e+04,33750.000000,33750.000000,3.375000e+04,33750.000000,33750.000000
mean,0.028859,-1.517930e-16,-2.829539e-16,8.421247e-19,4.989748,4.989748,-4.255425e-13,0.448739,0.001867,-6.652785e-17,2.520000,15.959170,1.103183e-16,2.982667,6.407674
std,0.167413,1.000000e+00,1.000000e+00,1.000000e+00,2.570021,2.570021,1.000000e+00,53.526500,0.043165,1.000000e+00,2.739005,9.081681,1.000000e+00,2.005717,3.543241
min,0.000000,-1.729749e+00,-3.454080e+00,-1.643256e+00,1.000000,1.000000,-2.087695e+01,0.000000,0.000000,-1.298084e+00,0.000000,0.000000,-1.626984e+00,0.000000,1.000000
25%,0.000000,-8.579564e-01,-6.651342e-01,-6.703950e-01,3.000000,3.000000,-6.223949e-01,0.000000,0.000000,-1.272359e+00,0.000000,7.000000,-1.041869e+00,1.000000,3.000000
50%,0.000000,1.893794e-03,4.693692e-02,-1.839644e-01,5.000000,5.000000,5.994111e-03,0.000000,0.000000,3.483563e-01,1.000000,17.000000,4.764371e-02,3.000000,6.000000
75%,0.000000,8.736864e-01,8.183473e-01,7.888966e-01,7.000000,7.000000,7.864488e-01,0.000000,0.000000,1.017223e+00,5.000000,24.000000,8.465441e-01,5.000000,10.000000
max,1.000000,1.721594e+00,1.708436e+00,6.139632e+00,9.000000,9.000000,2.099879e+00,9831.300000,1.000000,1.274479e+00,10.000000,37.000000,2.028575e+00,6.000000,12.000000


In [9]:
df.columns

Index([u'date_time', u'is_holiday', u'air_pollution_index', u'humidity',
       u'wind_speed', u'wind_direction', u'visibility_in_miles', u'dew_point',
       u'temperature', u'rain_p_h', u'snow_p_h', u'clouds_all',
       u'weather_type', u'weather_description', u'traffic_volume',
       u'dayofweek', u'month', u'hour'],
      dtype='object')

In [10]:
#saving file for easy training purpose
print('Saving as CSV...')
df.to_csv('clean_train.csv',index=False)
df.info()

Saving as CSV...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33750 entries, 0 to 33749
Data columns (total 18 columns):
date_time              33750 non-null datetime64[ns]
is_holiday             33750 non-null int64
air_pollution_index    33750 non-null float64
humidity               33750 non-null float64
wind_speed             33750 non-null float64
wind_direction         33750 non-null category
visibility_in_miles    33750 non-null int64
dew_point              33750 non-null int64
temperature            33750 non-null float64
rain_p_h               33750 non-null float64
snow_p_h               33750 non-null int64
clouds_all             33750 non-null float64
weather_type           33750 non-null int64
weather_description    33750 non-null int64
traffic_volume         33750 non-null float64
dayofweek              33750 non-null int64
month                  33750 non-null int64
hour                   33750 non-null category
dtypes: category(2), datetime64[ns](1), float64(7), i

In [ ]:
#running model
configs = json.load(open('config.json', 'r'))
if not os.path.exists(configs['model']['save_dir']): os.makedirs(configs['model']['save_dir'])

data = DataLoader(
    configs['data']['filename'],
    configs['data']['train_test_split'],
    configs['data']['columns']
)

model = Model()
model.build_model(configs)
#     model.load_model('./model.h5')

# out-of memory generative training
steps_per_epoch = math.ceil((data.len_train - configs['data']['sequence_length']) / configs['training']['batch_size'])
model.train_generator(
    data_gen=data.generate_train_batch(
        seq_len=configs['data']['sequence_length'],
        batch_size=configs['training']['batch_size'],
        normalise=configs['data']['normalise']
    ),
    epochs=configs['training']['epochs'],
    batch_size=configs['training']['batch_size'],
    steps_per_epoch=steps_per_epoch,
    save_dir=configs['model']['save_dir']
)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 24, 30)            5760      
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 30)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 24, 40)            11360     
_________________________________________________________________
lstm_3 (LSTM)                (None, 40)                12960     
_________________________________________________________________
dropout_2 (Dropout)          (None, 40)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 41        
Total params: 30,121
Trainable params: 30,121
Non-trainable params: 0
_________________________________________________________________
[Model